# Cleaning and analysis of German company data from Kaggle

## 1.Introduction 
- This project demonstrates the application of skills in cleaning, preparing, analyzing and visualizing data using Python and its libraries
- The data source is the website https://www.kaggle.com/
- Purpose: cleaning and preparation of csv file reading data using the pandas library, performing analysis and visualization using the numpy, matplotlib, seaborn, sklearn libraries

## 2. Import Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import numpy as np

## 3. Downloading data

In [ ]:
path = '../data/raw/Top_12_German_Companies_Financial_Data.csv'
df = pd.read_csv(path)

## 4.Data cleaning

In [ ]:
#check list of general data
#1. Size of data
print('Shape: ', df.shape)

In [ ]:
#2. Names of columns
print('Columns: ', df.columns.tolist())

In [ ]:
#3. Let's see first 10 rows
print(df.head(10))

In [ ]:
#4. And data types
print('\nData types: ')
print(df.dtypes)

In [ ]:
#5. Final lets prepare for cleaning , let's see a number of missing values
print('\nMissing values per column: ')
print(df.isnull().sum())

In [ ]:
print('How much duplicated in DataFrame: ',df.duplicated().sum())
print('\nMissing values in DataFrame:\n ',df.isnull().sum())

In [ ]:
#lets see Names of 12 Companies:
print('\nCompanies: ', df['Company'].unique())

At this stage, we see that data cleaning is not necessary because there are no missing values ​​or duplicates, all columns have information and the corresponding type, in which the information columns are of type int64 or float64

## 5.Basic Analyse and visualisation

In [ ]:
print('-------Basic Statistic------')
print(df.describe())

First of all lets see Leaders in Revenue values

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data = df ,x='Company', y = 'Revenue', hue = 'Company', palette = 'viridis')
plt.title('Revenue comparison by company')
plt.xlabel('Companies')
plt.ylabel('Revenue (EUR)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

Now lets analyse Profit comparison by companies

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=df, x='Company', y = 'Net Income', hue = 'Company', palette='viridis')
plt.title('Profit comparison by companies')
plt.xlabel('Companies')
plt.ylabel('Profit')
plt.xticks(rotation = 90)
plt.tight_layout()
plt.show()

Now lets analyse Liabilities comparison by companies

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=df, x='Company', y = 'Liabilities', hue = 'Company', palette='viridis')
plt.title('Liabilities comparison by companies')
plt.xlabel('Companies')
plt.ylabel('Liabilities')
plt.xticks(rotation = 90)
plt.tight_layout()
plt.show()

Now lets analyse Debt to Equity comparison by companies

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=df, x='Company', y = 'Debt to Equity', hue = 'Company', palette='viridis')
plt.title('Debt to Equity comparison by companies')
plt.xlabel('Companies')
plt.ylabel('Debt to Equity')
plt.xticks(rotation = 90)
plt.tight_layout()
plt.show()

Now lets analyse Equity comparison by companies

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=df, x='Company', y = 'Equity', hue = 'Company', palette='viridis')
plt.title('Equity comparison by companies')
plt.xlabel('Companies')
plt.ylabel('Equity')
plt.xticks(rotation = 90)
plt.tight_layout()
plt.show()

lets understand standard correlation between finance values
And take just numerical values

In [ ]:
df_num = df.select_dtypes(include=['int64','float64'])

plt.figure(figsize=(10,6))
sns.heatmap(df_num.corr(),annot=True, cmap='coolwarm')
plt.title('Correlation between finance values')
plt.show()

The -1 to 1 correlation coefficient measures the strength and direction of the linear relationship between two variables. A value of +1 indicates a perfect positive correlation (an increase in one variable is accompanied by an increase in the other), -1 indicates a perfect negative correlation
(an increase in one variable is accompanied by a decrease in the other), and values close to 0 indicate no linear relationship.

## 6.Deep analyse and prediction (Linear Regression), find leader by score (normalization)

Top leaders in financial score
Take a financial columns

In [ ]:
df_unique = df.groupby('Company')[['Revenue','Net Income','Equity','Assets','Liabilities','ROA (%)','ROE (%)','Debt to Equity']].mean().reset_index()
fin_cols = ['Revenue','Net Income','Equity','Assets','Liabilities','ROA (%)','ROE (%)','Debt to Equity']

Make a Normalization (this is done so that financial indicators are highly differentiated, and normalization will make it possible to compare them and carry out manipulations to determine positive and negative factors)

In [ ]:
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_unique[fin_cols]), columns=fin_cols)

Make a score for Companies : Take values like Revenue,Net Income,Equity,Assets,ROA,ROE as positive points and Liabilities and Debt to Equity as Negative

In [ ]:
df_scaled['Score'] = (df_scaled['Revenue']+df_scaled['Net Income']+df_scaled['Equity']+df_scaled['Assets']+
                      df_scaled['ROE (%)']+df_scaled['ROA (%)'] - df_scaled['Liabilities']-df_scaled['Debt to Equity'])

Add score to original df

In [ ]:
df_unique['Score'] = df_scaled['Score']

And show top by sorting with score value

In [ ]:
top_leaders = df_unique.sort_values(by='Score',ascending=False)
print('Leaders by score:')
print(top_leaders[['Score','Company','Revenue','Net Income','Equity','Assets','Liabilities','ROA (%)','ROE (%)','Debt to Equity']].head(12))

Visualization for leaders

In [ ]:
plt.figure(figsize = (10,7))
sns.barplot(data=df_unique, x = 'Company', y = 'Score', hue = 'Company', palette = 'viridis')
plt.xticks(rotation = 90)
plt.title('Leaders by score')
plt.xlabel('Company')
plt.ylabel('Score')
plt.show()

Try to show prediction for Revenue in next year for Companies (Linear Regression)

Lets change column period to year from DD/MM/YYYY to YYYY

In [ ]:
df['Year'] = pd.to_datetime(df['Period']).dt.year

Create list for prediction

In [ ]:
prediction = []

And with model as Linear regression lets predict a values

In [ ]:
for company in df['Company'].unique():
    company_df = df[df['Company']==company].sort_values('Year')
    X = company_df['Year'].values.reshape(-1,1)
    y = company_df['Revenue'].values
    #model creating
    model = LinearRegression()
    model.fit(X,y)
    next_year = np.array([[X.max() + 1]])
    pred = model.predict(next_year)[0]
    prediction.append({'Company': company,'Predicted_Revenue_next_year':pred})

pred_df = pd.DataFrame(prediction).sort_values(by='Predicted_Revenue_next_year', ascending=False)
print(pred_df.head(12))

Show a prediction for each company with line plot

In [ ]:
plt.figure(figsize=(15,6))
companies = df['Company'].unique()
colors = sns.color_palette('tab10', n_colors=len(companies))

for company in companies:
    numeric_cols = ['Revenue', 'Net Income', 'Liabilities', 'Assets', 'Equity', 'ROA (%)', 'ROE (%)', 'Debt to Equity']
    company_df = df[df['Company'] == company].groupby('Year')[numeric_cols].mean().reset_index().sort_values('Year')
    plt.plot(company_df['Year'], company_df['Revenue'], marker='o', label=company)
    next_year = company_df['Year'].max() + 1
    pred = pred_df.loc[pred_df['Company'] == company, 'Predicted_Revenue_next_year'].values[0]
    plt.scatter(next_year, pred, marker='X', s=100)

plt.xlabel('Year')
plt.ylabel('Revenue')
plt.title('Revenue and predict for next year')

plt.grid(True, axis='y', linestyle='--', alpha=0.6, color='gray')
plt.tight_layout()
plt.legend(bbox_to_anchor=(0.1, 0.51))
plt.show()

Try to show prediction for Net Income in next year for Companies (Linear Regression) - the same method
just change column from Revenue to Net Income (algorithm is the same)

In [ ]:
prediction = []
for company in df['Company'].unique():
    company_df = df[df['Company']==company].sort_values('Year')
    X = company_df['Year'].values.reshape(-1,1)
    y = company_df['Net Income'].values
    #model creating
    model = LinearRegression()
    model.fit(X,y)
    next_year = np.array([[X.max() + 1]])
    pred = model.predict(next_year)[0]
    prediction.append({'Company': company,'Predicted_Net_Income_next_year':pred})

pred_df = pd.DataFrame(prediction).sort_values(by='Predicted_Net_Income_next_year', ascending=False)
print(pred_df.head(12))

#Show a prediction for each company with line plot

plt.figure(figsize=(15,6))
companies = df['Company'].unique()
colors = sns.color_palette('tab10', n_colors=len(companies))

for company in companies:
    numeric_cols = ['Revenue', 'Net Income', 'Liabilities', 'Assets', 'Equity', 'ROA (%)', 'ROE (%)', 'Debt to Equity']
    company_df = df[df['Company'] == company].groupby('Year')[numeric_cols].mean().reset_index().sort_values('Year')
    plt.plot(company_df['Year'], company_df['Net Income'], marker='o', label=company)
    next_year = company_df['Year'].max() + 1
    pred = pred_df.loc[pred_df['Company'] == company, 'Predicted_Net_Income_next_year'].values[0]
    plt.scatter(next_year, pred, marker='X', s=100)

plt.xlabel('Year')
plt.ylabel('Net Income')
plt.title('Revenue and predict for next year')

plt.grid(True, axis='y', linestyle='--', alpha=0.6, color='gray')
plt.tight_layout()
plt.legend(bbox_to_anchor=(0.1, 0.51))
plt.show()